<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:Pablo Esteban Zúñiga Varela
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/PabloZV/Laboratorio-de-Programacion-Cientifica-para-Ciencia-de-Datos`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Utilizar la API de Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Utilizar `cache_lru` para cachear resultados de funciones.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Medir el tiempo de ejecución como también el uso de memoria de la función anterior.
- Optimizar la función anterior a través de `numba`.

#Importamos librerias utiles 😸

In [6]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%%capture
!pip install praw
!pip install line_profiler
!pip install numba

%load_ext autoreload
%autoreload 2

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://preview.redd.it/3sxusxcjge791.png?auto=webp&s=88bc1f9a3c59eafda24b0e32dd87d7cf596d205d">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [X] Estudiar la función propuesta por el equipo docente.
- [X] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [X] Estudiar la memoria ocupada por el código a través de un perfilador.
- [X] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [10]:
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

%%file praw_reddit.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url



UsageError: Line magic function `%%file` not found.


In [11]:
%%time
votes, post, url = praw_reddit()

CPU times: user 248 ms, sys: 6.3 ms, total: 254 ms
Wall time: 19.2 s


In [89]:
#Analizamos tiempos de ejecución
%reload_ext memory_profiler
%reload_ext line_profiler
%lprun -f praw_reddit praw_reddit()

Timer unit: 1e-06 s

Total time: 17.3982 s
File: /home/pablo/Documents/U/Lab9/praw_reddit.py
Function: praw_reddit at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     5         2        663.0    331.5      0.0      reddit = praw.Reddit(
     6         1          0.0      0.0      0.0          client_id="-w2hyFINxZ8T3g",
     7         1          1.0      1.0      0.0          client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     8         1          1.0      1.0      0.0          password="ClasesMDS7202",
     9         1          1.0      1.0      0.0          user_agent="Clases",
    10         1          0.0      0.0      0.0          username="DocenciaDataScience",
    11         1          1.0      1.0      0.0          check_for_async=False,
    12                                               )
    13         1         21.0     21.0      0.0      subr

In [90]:
#analizamos memoria
%reload_ext memory_profiler
%reload_ext line_profiler
from praw_reddit import praw_reddit
%mprun -f praw_reddit praw_reddit()


Filename: /home/pablo/Documents/U/Lab9/praw_reddit.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     4    259.2 MiB    259.2 MiB           1   def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     5    259.2 MiB      0.0 MiB           2       reddit = praw.Reddit(
     6    259.2 MiB      0.0 MiB           1           client_id="-w2hyFINxZ8T3g",
     7    259.2 MiB      0.0 MiB           1           client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     8    259.2 MiB      0.0 MiB           1           password="ClasesMDS7202",
     9    259.2 MiB      0.0 MiB           1           user_agent="Clases",
    10    259.2 MiB      0.0 MiB           1           username="DocenciaDataScience",
    11    259.2 MiB      0.0 MiB           1           check_for_async=False,
    12                                             )
    13    259.2 MiB      0.0 MiB           1       subreddit = reddit.subreddit(nombre_subreddit)
    14                                         
 

**Respuesta:**

    Notamos que el proceso que consume más tiempo es subreddit.hot(limit=n_hot), y lo hace significativamente más que los demás comandos
    Respecto a la memoria utilizada, notamos que usamos la misma cantidad de memoria todo el tiempo, la ejecución de la función termina con la misma memoria con la que se inicializa. esto puede ser porque la memoria que ocupa en cad aproceso no alacnza a sumar 0.01MB así que no alcanza a observarse a causa de la aproximación de la librería.

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [X] Responder las preguntas.
- [X] Mejorar el código con cache.
- [X] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

    1) Caché se le llama a cualquier memoria creada con el fin de optimizar algún proceso, y no ser el output principal del proceso en si. Las siglas LRU se refiere a least-recently-used items. Es un tipo de memoria caché que guarda los últimos n objetos utilizados. En pyrhon una implementación está en la librería functools bajo el nombre de lru_cache.
    2) Toma más memoria, y dependiendo del tipo de memoria, podría hasta requerir más procesamiento (por ejemplo si la memoria se va modificando continuamente y eso requiere un procesamiento), lo que uaríá más recursos del computador (procesamiento y memoria) a cambio de menores tiempos de ejecusión.
    3) Se podría guardar el resultado en RAM de la operación que más demora para que vaya más rápido.

In [108]:
#declaramos la función original
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url



In [112]:
%%timeit
#medimos cuanto demora
praw_reddit()

18.4 s ± 1.05 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [100]:
import praw
import numpy as np

@lru_cache(maxsize=100)
def cached_praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [103]:
%%timeit
#corremos el código una primera vez para armar el cache
cached_praw_reddit()

The slowest run took 16.16 times longer than the fastest. This could mean that an intermediate result is being cached.
256 ns ± 404 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [104]:
%%timeit
#ejecutamos el código una segunda vez para ver el tiempo de ejecución usando el caché
cached_praw_reddit()

26.3 ns ± 0.898 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


**Respuesta de lru_cache sobre praw_reddit**

    Se observa que la función pasa de demorarse 18.4 segundos a demorarse 26.3 nano segundos, es un aumento en la 

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [12]:
#vemos la forma de los datos a procesar
print(votes[0].shape)
print(votes[1].shape)
print(votes.keys())
print(votes[2].shape)
print(votes[3].shape)
print(votes)
print(votes[0])
print(votes[0][1])

(50, 2)
(100, 2)
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])
(150, 2)
(200, 2)
{0: array([[  0,   0],
       [154,  12],
       [ 76,   2],
       [ 70,  14],
       [361,  11],
       [ 62,  11],
       [ 37,   9],
       [ 32,   4],
       [ 15,   1],
       [ 50,   8],
       [ 69,   5],
       [194,   8],
       [ 25,   2],
       [ 82,   2],
       [ 67,   3],
       [ 14,   2],
       [ 59,   4],
       [142,   4],
       [ 20,   1],
       [ 15,   3],
       [ 90,   7],
       [ 58,   3],
       [562,  77],
       [ 59,   4],
       [ 51,   4],
       [286,  43],
       [420,  68],
       [185,  38],
       [  6,   2],
       [120,   4],
       [ 71,   2],
       [ 38,   4],
       [133,  52],
       [160,  24],
       [276,   3],
       [161,  22],
       [ 62,   5],
       [160,  12],
       [  4,   2],
       [152,  19],
       [133,  12],
       [ 77,  15],
       [ 37,  19],
       [ 27,  15],
       [ 32,   1],
       [  8,   2],
       [136, 

In [13]:
import math
def without_numpy_mean_posterior_and_standrard_error(all_u,all_d)->tuple[np.array,np.array]:
    mus=[]
    sigmas=[]
    for u,d in zip(all_u,all_d):
        a=1+u
        b=1+d
        sigma=1.65*math.sqrt(a*b/((a+b)**2*(a+b+1)))
        mu=a/(a+b)
        mus.append(mu)
        sigmas.append(sigma)
    mus=np.array(mus)
    sigmas=np.array(sigmas)
    return mus,sigmas
def mean_posterior_and_standrard_error(all_u,all_d)->tuple[np.array,np.array]:
    a=1+all_u
    b=1+all_d
    sigma=1.65*np.sqrt(a*b/((a+b)**2*(a+b+1)))
    mu=a/(a+b)
    return mu,sigma

## 1.4 Comparación de Rendimiento [2 Puntos]

Ahora, aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

Para esto genere un gráfico de linea utilizando plotly, en donde se pueda observar el tiempo que toma ejecutar cada función (el eje y) diferentes cantidades de datos (ejemplo, en el eje x considerar 100, 200, 300, ..., 1000 datos) sobre cada función implementada (series `Python`, `Numpy`, `Python-JIT`, `Numpy-JIT`). ¿Es posible observar diferencias? ¿A qué se debe esto?.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

**Nota 2**: En el caso que su computador sea muy rápido, se recomienda graficar el eje y (tiempo) como logaritmo usando `log_y=True`.


In [57]:
import plotly.graph_objects as go
from numba import jit
import math
#creamos las funciones anteriores pero usando numba
@jit(nopython=True)
def numba_without_numpy_mean_posterior_and_standrard_error(all_u,all_d):
    mus=[]
    sigmas=[]
    for i in  range(len(all_u)):
        u=all_u[i]
        d=all_d[i]
        a=1+u
        b=1+d
        sigma=1.65*math.sqrt(a*b/((a+b)**2*(a+b+1)))
        mu=a/(a+b)
        mus.append(mu)
        sigmas.append(sigma)
    return mus,sigmas

@jit(nopython=True)
def numba_mean_posterior_and_standrard_error(all_u,all_d):
    a=1+all_u
    b=1+all_d
    sigma=1.65*np.sqrt(a*b/((a+b)*(a+b)*(a+b+1)))
    mu=a/(a+b)
    return mu,sigma

#creamos la data a graficar
times_without_numpy_mean_posterior_and_standrard_error=[]
times_with_numpy_mean_posterior_and_standrard_error=[]
times_numba_without_numpy_mean_posterior_and_standrard_error=[]
times_numba_with_numpy_mean_posterior_and_standrard_error=[]
X=[]
for i in range(len(votes)):
    time_without_numpy = %timeit -o without_numpy_mean_posterior_and_standrard_error(votes[i][:,0],votes[i][:,1])
    times_without_numpy_mean_posterior_and_standrard_error.append(time_without_numpy.average)
    time_with_numpy = %timeit -o mean_posterior_and_standrard_error(votes[i][:,0],votes[i][:,1])
    times_with_numpy_mean_posterior_and_standrard_error.append(time_with_numpy.average)
    time_numba_without_numpy = %timeit -o numba_without_numpy_mean_posterior_and_standrard_error(votes[i][:,0],votes[i][:,1])
    times_numba_without_numpy_mean_posterior_and_standrard_error.append(time_numba_without_numpy.average)
    time_numba_with_numpy = %timeit -o numba_mean_posterior_and_standrard_error(votes[i][:,0],votes[i][:,1])
    times_numba_with_numpy_mean_posterior_and_standrard_error.append(time_numba_with_numpy.average)
    X.append(len(votes[i]))

fig = go.Figure([
    go.Scatter(name='times_without_numpy_mean_posterior_and_standrard_error',x=X,y=times_without_numpy_mean_posterior_and_standrard_error,mode='lines'),
    go.Scatter(name='times_with_numpy_mean_posterior_and_standrard_error',x=X,y=times_with_numpy_mean_posterior_and_standrard_error,mode='lines'),
    go.Scatter(name='times_numba_without_numpy_mean_posterior_and_standrard_error',x=X,y=times_numba_without_numpy_mean_posterior_and_standrard_error,mode='lines'),
    go.Scatter(name='times_numba_with_numpy_mean_posterior_and_standrard_error',x=X,y=times_numba_with_numpy_mean_posterior_and_standrard_error,mode='lines')
])
    
fig.show()

41.4 µs ± 1.35 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
7.23 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
1.8 µs ± 28.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
1.61 µs ± 16.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
82.8 µs ± 2.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
7.51 µs ± 97.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.31 µs ± 152 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
1.74 µs ± 25.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
120 µs ± 932 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
8.86 µs ± 95.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
4.49 µs ± 38.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.01 µs ± 26 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
163 µs ± 6.77 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
9.03 µs ± 195 ns

    Se observa que tanto usar numpy como usar numpy como numba mejora la velocidad de procesamiento, siendo la utilizacion de numba lo que más aporta en este sentido. En esta misma linea, usando tanto numpy como numba se obtienen los mejores resultados. Esto porque  (1) al usar numpy se utilizan versiones más optimizadas de las funciones, que corren en un lenguaje de programción de más bajo nivel que python, y (2) porque se precompila python en vez de ser interpretado como es regularmente, dando cabida a optimizaciones en el código compilado que en su versión interpretada no había.  

## 1.5 Plot de Resultados de Análisis de Votos [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [55]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = mean_posterior_and_standrard_error(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



In [56]:
r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

**Respuesta:**


    Se observa que los post con mayor media suelen tener menor desviación estandar. Esto se puede interpretar como que aquello que es descrito más positivamente es menos criticado, mientras que es más fácil encontrar opiniocnes diferentes cuando la laloración de un post no es tan buena.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>